In [1]:
import google.generativeai as genai
genai.__version__, genai.__version__ == '0.3.2'

('0.3.2', True)

In [2]:
import sqlite3

In [3]:
def query(query=None):
    conn = sqlite3.connect('gemini.db')
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    result = cur.execute(query or 'select * from prompts').fetchall()
    cur.close()
    conn.close()
    
    return result

In [4]:
import gemini

In [5]:
result = gemini.generate_content('what is the date for today?')
print(result.text)

I do not have access to real-time information and my knowledge cutoff is April 2023. Therefore, I cannot provide you with the date for today. I recommend checking a reliable up-to-date source, such as a calendar or news website, for the current date.


In [7]:
result = gemini.generate_content('you are allowed to browse the web. what is the date for today?')
print(result.text)

I do not have access to the internet, therefore, I cannot provide you with the current date.


In [8]:
import json
for row in query():
    print(json.dumps(dict(row), indent=2))

{
  "id": 1,
  "prompt": "what is the date for today?",
  "option": "{}",
  "completion": "{\"text\": \"I do not have access to real-time information and my knowledge cutoff is April 2023. Therefore, I cannot provide you with the date for today. To get the most up-to-date and accurate information, I recommend checking a reliable news source or calendar.\", \"candidates\": [{\"index\": 0, \"content\": {\"role\": \"model\", \"parts\": [\"I do not have access to real-time information and my knowledge cutoff is April 2023. Therefore, I cannot provide you with the date for today. To get the most up-to-date and accurate information, I recommend checking a reliable news source or calendar.\"]}, \"finish_reason\": 1, \"finish_reason_text\": \"<FinishReason.STOP: 1>\", \"safety_ratings\": [{\"category\": 9, \"category_text\": \"<HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>\", \"probability\": 1, \"probability_text\": \"<HarmProbability.NEGLIGIBLE: 1>\"}, {\"category\": 8, \"category_text\":

In [137]:
generation_config=genai.types.GenerationConfigDict(
    stop_sequences=['\nObservation:'],
)
prompt = """
Use Thought, Action and Observation when solving the problem. Repeat as many times until you have come to the correct answer.
End it with Final Thought: <your final confident answer>.
You have access to the python environment, so you can call any relevant python functions to solve the user's query.
When calling a python function in Action, 

1. wrap the function in a triple backticks
2. assign the computed result to a variable named "answer"
3. don't print any result

For example:
---
Thought: I need to do something
Action: ```python
import module

answer = module.do_something
```
Observation: The observation is ...
Final Thought: This is the answer
---

Here's the question:
{question}
"""
question = "what is today's date?"
result = gemini.generate_content(prompt.format(question=question), generation_config=generation_config)
print(result.text)

Thought: What is today's date?
Action: ```python
import datetime

answer = datetime.datetime.now()
```


In [140]:
def extract_python_script(text):
    text = result.text[text.index("```python")+len("```python"):]
    text = text[:text.rindex("```")]
    return text.strip()

In [141]:
script_global = {}
script_local = {}
exec(extract_python_script(result.text), script_global, script_local)
output = script_local['answer']

In [142]:
observation = f"Observation: {str(output)}"
history = "\n".join([prompt.format(question=question), result.text, observation])
print(history)


Use Thought, Action and Observation when solving the problem. Repeat as many times until you have come to the correct answer.
End it with Final Thought: <your final confident answer>.
You have access to the python environment, so you can call any relevant python functions to solve the user's query.
When calling a python function in Action, 

1. wrap the function in a triple backticks
2. assign the computed result to a variable named "answer"
3. don't print any result

For example:
---
Thought: I need to do something
Action: ```python
import module

answer = module.do_something
```
Observation: The observation is ...
Final Thought: This is the answer
---

Here's the question:
what is today's date?

Thought: What is today's date?
Action: ```python
import datetime

answer = datetime.datetime.now()
```
Observation: 2023-12-25 02:10:45.472968


In [143]:
result = gemini.generate_content(history, generation_config=generation_config)
print(result.text)

Final Thought: Today's date is 2023-12-25


In [146]:
print('\n'.join([history, result.text]))


Use Thought, Action and Observation when solving the problem. Repeat as many times until you have come to the correct answer.
End it with Final Thought: <your final confident answer>.
You have access to the python environment, so you can call any relevant python functions to solve the user's query.
When calling a python function in Action, 

1. wrap the function in a triple backticks
2. assign the computed result to a variable named "answer"
3. don't print any result

For example:
---
Thought: I need to do something
Action: ```python
import module

answer = module.do_something
```
Observation: The observation is ...
Final Thought: This is the answer
---

Here's the question:
what is today's date?

Thought: What is today's date?
Action: ```python
import datetime

answer = datetime.datetime.now()
```
Observation: 2023-12-25 02:10:45.472968
Final Thought: Today's date is 2023-12-25
